In [1]:
pip install tsplib95

In [50]:
import tsplib95
problem = tsplib95.load("/content/bayg29.tsp")
#cities = list(problem.get_nodes());

In [92]:
import random
#GNOME_SIZE = len(list(problem.get_nodes()))
#POPULATION_SIZE = 100;

initially randomly populate the population. we use twice the population size(because we cut half of it for breeding).

In [178]:
def populate(cities, POPULATION_SIZE):
  GNOME_SIZE = len(cities)
  population = []
  
  for i in range(0, 2 * POPULATION_SIZE):
    population.append(random.sample(cities, GNOME_SIZE))
  
  return population

fitness calculates the cost of a given gnome. a gnome with less fitness is a better gnome.

In [177]:
def fitness(gnome, dataset):
  GNOME_SIZE = len(gnome)
  fit = 0
  for i in range(GNOME_SIZE):
    l = gnome[i]
    r = gnome[(i + 1) % GNOME_SIZE]
    fit += dataset.get_weight(l, r)
  return fit

this function calculates the fitness of a given population

In [176]:
def calculate_fitness(population, dataset):
  fit = []
  for i in range(len(population)):
    fit.append(fitness(population[i], dataset))
  return fit

this function recieves a populations and gives it back sorted based on its fitness

In [175]:
def sort_fitness(population, dataset):
  fit = calculate_fitness(population, dataset);
  sorted_pair = sorted(zip(fit, population))
  ans = []
  for i in range(len(sorted_pair)):
    ans.append(sorted_pair[i][1])
  return ans

we select the top ranked members of the population for breeding

In [174]:
def selection(population, ELITE_SIZE, dataset):
  population = sort_fitness(population, dataset)
  return population[:ELITE_SIZE]

this function outputs the offspring of 2 gnomes based on ordered crossover (OC)

In [151]:
def single_crossover(parent1, parent2):
  GNOME_SIZE = len(parent1)
  l = random.randint(0, GNOME_SIZE)
  r = random.randint(0, GNOME_SIZE)
  l, r = min(l, r), max(l, r)
  
  child = [0] * GNOME_SIZE

  for i in range(l, r):
    child[i] = parent1[i]
  
  j = 0
  for i in range(0, GNOME_SIZE):
    if(j == l):
      j = r
    if(not(parent2[i] in child)):
      child[j] = parent2[i]
      j += 1

  return child
  

this function performs crossover over the whole population.

In [93]:
def crossover(population, POPULATION_SIZE):
  children = []
  for i in range(POPULATION_SIZE):
    p1 = random.randint(0, len(population) - 1)
    p2 = random.randint(0, len(population) - 1)
    children.append(single_crossover(population[p1], population[p2]))

  return children


this function mutates a gnome based on mutation rate. we use swap mutation method.

In [97]:
def mutate(gnome, mutation_rate):
  GNOME_SIZE = len(gnome)
  for i in range(GNOME_SIZE):
    random_pos = random.randint(0, GNOME_SIZE - 1)
    random_pick = random.random()
    if(random_pick <= mutation_rate):
      gnome[i], gnome[random_pos] = gnome[random_pos], gnome[i]
      break
  return gnome

this function mutates the whole population of children created by crossover

In [95]:
def mutate_population(population, mutation_rate):
  for i in range(len(population)):
    population[i] = mutate(population[i], mutation_rate)

  return population


this is the main part of our genetic algorithm where we first create an initial population and then in every cycle we first select a subset of the population and then we use crossover on that subset and then we mutate the children newly created and we add those to the parents that we already had and we do this cycle for ITERATION_COUNT times

In [181]:
POPULATION_SIZE = 2000
ITERATION_COUNT = 400
MUTATION_RATE = 1/30
cities = list(problem.get_nodes());
population = populate(cities, POPULATION_SIZE)

for i in range(ITERATION_COUNT):
  print(i)
  population = selection(population, POPULATION_SIZE, problem)
  print(fitness(population[0], problem))
  children = crossover(population, POPULATION_SIZE)
  children = mutate_population(children, MUTATION_RATE)
  population.extend(children)

0
3413
1
3413
2
3338
3
3263
4
3163
5
2941
6
2851
7
2851
8


KeyboardInterrupt: ignored

first run: population size = 500 iteration: 500 mutation-rate= 1/30 best-cost = 1635

second run: population size = 500 iteration= 200 mutation rate = 1/20 best-cost = 1654

third run: population size = 200 iteration = 200 mutation rate = 1/20 best cost = 1663

4th run: population size = 200 iteration = 200 mutation rate = 1/30 best cost = 1709

5th run: population size = 200 iteration = 500 mutation rate = 1/30 best cost = 1615 converged after 200 iterations

6th run: identical to 4th run best cost = 1646

7th run: population size = 200 iteration = 400 mutation rate = 1/10 best cost = 1755

8th run: population size = 200 iteration = 400 mutation rate = 1/40 best cost = 1747

9th run: population size = 400 iteration = 400 mutation rate = 1/40 best  cost = 1637 converged in 200 iterations

10th run: population size = 2000 iteration = 200 mutation rate = 1/30 best cost = 1620

conclusion: 200 iterations was nearly enough for all the runs for the answer to converge. lower mutation rate means faster converging to a local maximal point and making the population size bigger would fix that problem.
higher mutation rates means our algorithm runs more randomly and hense it converges later on and needs more iterations. and depending on luck it might never converge to a good answer we want.
all and all the answers to all the runs are fairly close to each other and the lowest cost obtained from the algorithm doesnt change that much by making fairly small changes to iteration count, population size and mutation rate.
and if we make our population size significantly bigger the algorithm would run much much lower than we need it to and considering the time aspect its not worth making the population size very large altough it might in theory give us a better answer.

by looking at our last run we see that the answer from a much larger population size is not at all better than a small population size but it took almost 10 times more to run the algorithm. so its not always good to have a large population size.